In [1]:
import pandas as pd
import numpy as np
import datetime
import quandl
import config


# settings to observe x amount of rows in dataframe
pd.set_option('max_rows', 500)
pd.set_option('max_columns', 1000)

# API Key
key = quandl.ApiConfig.api_key

In [2]:
tickers_data = pd.read_csv('/Users/Alfredo/Desktop/clustering-stocks/WIKI-datasets-codes.csv', header=-1)
tickers_data.columns = ['Ticker', 'Description']
tickers_data.head()

,Ticker,Description
0,WIKI/AAPL,"Apple Inc (AAPL) Prices, Dividends, Splits and..."
1,WIKI/AA,"Alcoa Inc. (AA) Prices, Dividends, Splits and ..."
2,WIKI/ABBV,"AbbVie Inc. (ABBV) Prices, Dividends, Splits a..."
3,WIKI/ABC,"AmerisourceBergen Corp. (ABC) Prices, Dividend..."
4,WIKI/ABT,"Abbott Laboratories (ABT) Prices, Dividends, S..."


In [3]:
# Loop through tickers given
tickers = tickers_data.Ticker[:3].tolist()

symbols = []

for sym in tickers:
    
    if sym in tickers:
        sym = sym.replace(".", "_")
    
    start = "2016-01-01"
    end = "2016-01-15"
    
    mydata = quandl.get(sym, start_date=start, end_date=end, api_key = key)
    mydata['Symbol'] = sym
    symbols.append(mydata)
    
    
# attach tickers tables
mydata = pd.concat(symbols)

# index 'Date' to column
mydata.reset_index(inplace=True)

# Change location of column 'Symbol'
symb = mydata['Symbol']
mydata.drop(labels=['Symbol'], axis=1,inplace = True)
mydata.insert(1, 'Symbol', symb)
mydata.head(3)

,Date,Symbol,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
0,2016-01-04,WIKI/AAPL,102.61,105.368,102.00,105.35,67649387.0,0.0,1.0,99.136516,101.801154,98.547165,101.783763,67649387.0
1,2016-01-05,WIKI/AAPL,105.75,105.850,102.41,102.71,55790992.0,0.0,1.0,102.170223,102.266838,98.943286,99.233131,55790992.0
2,2016-01-06,WIKI/AAPL,100.56,102.370,99.87,100.70,68457388.0,0.0,1.0,97.155911,98.904640,96.489269,97.291172,68457388.0


In [4]:
# Slices WIKI out of symbol column for tickers
mydata['Symbol'] = mydata['Symbol'].map(lambda x: str(x)[5:])
#mydata.head(3)

# resetting indexes
mydata = mydata.set_index(['Date','Symbol']).unstack(0).stack(dropna=False).groupby(level='Symbol').ffill().bfill()
mydata.head()

Open     High     Low   Close      Volume  Ex-Dividend  \
Symbol Date                                                                   
AAPL   2016-01-04  102.61  105.368  102.00  105.35  67649387.0          0.0   
       2016-01-05  105.75  105.850  102.41  102.71  55790992.0          0.0   
       2016-01-06  100.56  102.370   99.87  100.70  68457388.0          0.0   
       2016-01-07   98.68  100.130   96.43   96.45  81094428.0          0.0   
       2016-01-08   98.55   99.110   96.76   96.96  70798016.0          0.0   

                   Split Ratio   Adj. Open   Adj. High   Adj. Low  Adj. Close  \
Symbol Date                                                                     
AAPL   2016-01-04          1.0   99.136516  101.801154  98.547165  101.783763   
       2016-01-05          1.0  102.170223  102.266838  98.943286   99.233131   
       2016-01-06          1.0   97.155911   98.904640  96.489269   97.291172   
       2016-01-07          1.0   95.339552   96.740467  93.165717   93.185040   
       2016-01-08          1.0   95.213952   95.754996  93.484546   93.677776   

                   Adj. Volume  
Symbol Date                     
AAPL   2016-01-04   67649387.0  
       2016-01-05   55790992.0  
       2016-01-06   68457388.0  
       2016-01-07   81094428.0  
       2016-01-08   70798016.0

In [5]:
# Builds daily return column
mydata['Daily_Return'] = mydata['Adj. Close'].pct_change(1)
#mydata.head()

# Builds overall percentage on return or Cumulitive return(return difference from start to current price)
mydata['Cumulitive_Return'] = mydata.groupby('Symbol')['Adj. Close'].transform(lambda x: x[-1]/x[0] -1)
#mydata.head(15)

# Calculates daily average high, low, volume trader for each stock
mydata['Avg_High'] = mydata.groupby('Symbol').High.transform('mean')
mydata['Avg_Low'] = mydata.groupby('Symbol').Low.transform('mean')
mydata['Avg_Vol_Trader'] = mydata.groupby('Symbol').Volume.transform('mean')
mydata['Avg_Dividend'] = mydata.groupby('Symbol')['Ex-Dividend'].transform('mean')


# Mean Daily Return
d_mean_return = mydata.groupby('Symbol').Daily_Return.transform('mean').to_frame()
mydata['Avg_Daily_Return'] = d_mean_return

# Std Daily
d_std_return = mydata.groupby('Symbol').Daily_Return.transform('std').to_frame()
mydata['Std_Daily_Return'] = d_std_return

# Calculates Daily Sharpe Ratio(Calculates risk adjusted return)
# mean return - risk free rate / standard deviation
# For this formula we are assuming a risk free rate of 0(different people,different opinions)
sr = d_mean_return/d_std_return
mydata['Daily_Sharpe_Ratio'] = sr


mydata.head(15)

Open     High       Low   Close      Volume  Ex-Dividend  \
Symbol Date                                                                     
AAPL   2016-01-04  102.61  105.368  102.0000  105.35  67649387.0          0.0   
       2016-01-05  105.75  105.850  102.4100  102.71  55790992.0          0.0   
       2016-01-06  100.56  102.370   99.8700  100.70  68457388.0          0.0   
       2016-01-07   98.68  100.130   96.4300   96.45  81094428.0          0.0   
       2016-01-08   98.55   99.110   96.7600   96.96  70798016.0          0.0   
       2016-01-11   98.97   99.060   97.3400   98.53  49739377.0          0.0   
       2016-01-12  100.55  100.690   98.8399   99.96  49154227.0          0.0   
       2016-01-13  100.32  101.190   97.3000   97.39  62439631.0          0.0   
       2016-01-14   97.96  100.480   95.7400   99.52  63170127.0          0.0   
       2016-01-15   96.20   97.710   95.3600   97.13  79833891.0          0.0   
ABBV   2016-01-04   58.06   58.320   56.4000   57.61  10308397.0          0.0   
       2016-01-05   57.99   58.250   56.8150   57.37   7179634.0          0.0   
       2016-01-06   56.37   57.560   56.1800   57.38   8952688.0          0.0   
       2016-01-07   56.52   57.840   56.3400   57.21   9292584.0          0.0   
       2016-01-08   57.55   57.810   55.4800   55.65   7985156.0          0.0   

                   Split Ratio   Adj. Open   Adj. High   Adj. Low  Adj. Close  \
Symbol Date                                                                     
AAPL   2016-01-04          1.0   99.136516  101.801154  98.547165  101.783763   
       2016-01-05          1.0  102.170223  102.266838  98.943286   99.233131   
       2016-01-06          1.0   97.155911   98.904640  96.489269   97.291172   
       2016-01-07          1.0   95.339552   96.740467  93.165717   93.185040   
       2016-01-08          1.0   95.213952   95.754996  93.484546   93.677776   
       2016-01-11          1.0   95.619735   95.706688  94.044912   95.194629   
       2016-01-12          1.0   97.146250   97.281511  95.494039   96.576222   
       2016-01-13          1.0   96.924036   97.764585  94.006267   94.093220   
       2016-01-14          1.0   94.643925   97.078619  92.499075   96.151117   
       2016-01-15          1.0   92.943503   94.402387  92.131938   93.842021   
ABBV   2016-01-04          1.0   53.870818   54.112058  52.330591   53.453287   
       2016-01-05          1.0   53.805869   54.047109  52.715648   53.230603   
       2016-01-06          1.0   52.302756   53.406894  52.126465   53.239882   
       2016-01-07          1.0   52.441933   53.666692  52.274921   53.082148   
       2016-01-08          1.0   53.397616   53.638856  51.476972   51.634706   

                   Adj. Volume  Daily_Return  Cumulitive_Return   Avg_High  \
Symbol Date                                                                  
AAPL   2016-01-04   67649387.0           NaN          -0.078026  101.19580   
       2016-01-05   55790992.0     -0.025059          -0.078026  101.19580   
       2016-01-06   68457388.0     -0.019570          -0.078026  101.19580   
       2016-01-07   81094428.0     -0.042205          -0.078026  101.19580   
       2016-01-08   70798016.0      0.005288          -0.078026  101.19580   
       2016-01-11   49739377.0      0.016192          -0.078026  101.19580   
       2016-01-12   49154227.0      0.014513          -0.078026  101.19580   
       2016-01-13   62439631.0     -0.025710          -0.078026  101.19580   
       2016-01-14   63170127.0      0.021871          -0.078026  101.19580   
       2016-01-15   79833891.0     -0.024015          -0.078026  101.19580   
ABBV   2016-01-04   10308397.0     -0.430391           0.006398   56.95896   
       2016-01-05    7179634.0     -0.004166           0.006398   56.95896   
       2016-01-06    8952688.0      0.000174           0.006398   56.95896   
       2016-01-07    9292584.0     -0.002963           0.006398   56.95896   
       2016-01-08

In [6]:
mydata1 = mydata.drop(mydata.columns[:14], axis=1)
mydata2 = mydata1.reset_index(level=1, drop=True)
mydata3 = mydata2.groupby('Symbol').first()
mydata3

,Avg_High,Avg_Low,Avg_Vol_Trader,Avg_Dividend,Avg_Daily_Return,Std_Daily_Return,Daily_Sharpe_Ratio
Symbol,,,,,,,
AAPL,101.19580,98.20499,64812746.4,0.000,-0.008744,0.023240,-0.376236
ABBV,56.95896,54.23150,11486529.4,0.057,-0.041795,0.141421,-0.295538
